In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("questions_dataset.csv")
questions = data["questions"].tolist()
answers = data["answers"].tolist()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

def get_response(input_question):
    input_question = [input_question]
    input_vector = vectorizer.transform(input_question)
    similarities = cosine_similarity(input_vector, X)
    closest = np.argmax(similarities)
    return answers[closest]



while True:
    user_question = input()
    response = get_response(user_question)
    print(response)
    if user_question == "Пока" or user_question == "Отмена":
        break
# tfidf

Добрый день, чем могу помочь?
Большое спасибо!
Браво 6, рубим свет.
Добрый день, чем могу помочь?
Сейчас у меня нет выхода в интернет, непременно задайте мне этот вопрос через годик!
Конечно! А вы не видели?
Очень жаль.
Очень жаль.
Очень здорово.
Очень здорово.
Хорошего дня!


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
import spacy
import pymorphy2
nltk.download('stopwords')
from nltk.corpus import stopwords
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

[nltk_data] Downloading package stopwords to C:\Users\Dmitriy
[nltk_data]     Drozhzhin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Dmitriy Drozhzhin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def preprocess_input(user_input):
    nlp = spacy.load("ru_core_news_sm")
    stopwords_ru = stopwords.words('russian')
    morph = pymorphy2.MorphAnalyzer()

    
    result = [morph.parse(token.text.lower())[0].normal_form for token in nlp(user_input)
               if not token.is_punct and token.text.lower() not in stopwords_ru]


    return result

def vectorize_input(processed_input):
    from transformers import BertTokenizer
    model_name = 'DeepPavlov/rubert-base-cased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    result = tokenizer.encode_plus(processed_input)['input_ids']

    return result


def postprocess_output(output_tensor):
    labels = answers[:]
    _, predicted_index = torch.max(output_tensor, dim=1)
    output_label = labels[predicted_index.item()]
    return output_label


class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(Transformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_dim, num_heads, hidden_dim, dropout),
            num_layers)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.linear(x)
        return x

num_epochs = 3
batch_size = 1 

while True:
    user_input = input()

    processed_input = preprocess_input(user_input)
    vectorized_input = vectorize_input(processed_input)

    input_tensor = torch.tensor(vectorized_input).unsqueeze(0)

    input_dim = len(vectorized_input)
    hidden_dim = len(vectorized_input)
    output_dim = len(vectorized_input)
    num_layers = 4 
    num_heads = 1 
    dropout = 0.2 

    model = Transformer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        input_tensor = torch.tensor(vectorized_input, dtype=torch.float).unsqueeze(0)
        output_tensor = model(input_tensor)
        
        optimizer.zero_grad()
        
        output = postprocess_output(output_tensor)

    print(output)
    if user_input == "Пока" or user_input == "Отмена":
        break
# transformers

c:\Users\Dmitriy Drozhzhin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Добрый день, чем могу помочь?


c:\Users\Dmitriy Drozhzhin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Хорошего дня!
Прекрасно, спасибо!
Прекрасно, спасибо!
Прекрасно, спасибо!
Хорошего дня!


In [10]:
data = pd.read_csv("questions_dataset.csv")
questions = data["questions"].tolist()
answers = data["answers"].tolist()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased')

def preprocess_input(user_input):
    inputs = tokenizer(user_input, return_tensors="pt")
    return inputs

def generate_response(input_tensor):
    outputs = model(**input_tensor)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return answers[predicted_label]

while True:
    user_question = input()
    input_tensor = preprocess_input(user_question)
    response = generate_response(input_tensor)
    print(response)
    if user_question == "Пока" or user_question == "Отмена":
        break
# hybrid


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Добрый день, чем могу помочь?
Добрый день, чем могу помочь?
Добрый день, чем могу помочь?
Добрый день, чем могу помочь?
